In [1]:
import requests
import pandas as pd
from datetime import datetime
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the environment variable
Token_MC_Smart = os.getenv("Token_MC_Smart")

In [2]:
# API endpoint
url = "https://api.moysklad.ru/api/remap/1.2/report/profit/byproduct"

# Parameters for the request
params = {
    "limit": 1000,  # Maximum number of entities to retrieve
    "offset": 0,     # Offset in the list of entities
    "momentFrom": datetime.now().strftime("%Y-01-01"),  # Beginning of the current year
    "momentTo": datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Current date and time
}

# Headers for the request
headers = {
    "Authorization": Token_MC_Smart,  # Replace with your access token
    "Content-Type": "application/json"
}

# Make the GET request
response = requests.get(url, headers=headers, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Display the DataFrame
    print(data)
else:
    print(f"Request failed with status code: {response.status_code}")
    print(response.text)

{'context': {'employee': {'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/context/employee', 'metadataHref': 'https://api.moysklad.ru/api/remap/1.2/entity/employee/metadata', 'type': 'employee', 'mediaType': 'application/json'}}}, 'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/report/profit/byproduct?limit=1000&offset=0&momentFrom=2025-01-01&momentTo=2025-06-06+17%3A35%3A43', 'type': 'salesbyproduct', 'mediaType': 'application/json', 'size': 90, 'limit': 1000, 'offset': 0}, 'rows': [{'assortment': {'meta': {'href': 'https://api.moysklad.ru/api/remap/1.2/entity/product/e430aa72-8ada-11ef-0a80-1006000ec7ba', 'metadataHref': 'https://api.moysklad.ru/api/remap/1.2/entity/product/metadata', 'type': 'product', 'mediaType': 'application/json', 'uuidHref': 'https://online.moysklad.ru/app/#good/edit?id=e430a07d-8ada-11ef-0a80-1006000ec7b5'}, 'name': 'Венчик для миксеров KitchenAid', 'code': '5K5A2WW_DM', 'article': '5K5A2WW_DM', 'uom': {'meta': {'href': 'https://api.moysklad.ru

In [3]:
import pandas as pd


def flatten_product_row(row):
    """Flatten a single row of product data, handling missing keys."""
    assortment = row.get('assortment', {})
    meta = assortment.get('meta', {})
    uom = assortment.get('uom', {}).get('name', None)
    
    flat_row = {
        # Product info
        'product_name': assortment.get('name'),
        'product_code': assortment.get('code'),
        'product_article': assortment.get('article'),
        'product_uom': uom,
        'product_href': meta.get('href'),
        'product_uuid': meta.get('uuidHref', '').split('id=')[-1] if meta.get('uuidHref') else None,
        
        # Sales metrics
        'sell_quantity': row.get('sellQuantity'),
        'sell_price': row.get('sellPrice'),
        'sell_sum': row.get('sellSum'),
        'sell_cost_sum': row.get('sellCostSum'),
        'return_quantity': row.get('returnQuantity'),
        'return_sum': row.get('returnSum'),
        'profit': row.get('profit'),
        'margin': row.get('margin'),
        'sales_margin': row.get('salesMargin')
    }
    return flat_row

# Process all rows
rows = [flatten_product_row(row) for row in data['rows']]

# Create DataFrame
df = pd.DataFrame(rows)

# Display the DataFrame
df

,product_name,product_code,product_article,product_uom,product_href,product_uuid,sell_quantity,sell_price,sell_sum,sell_cost_sum,return_quantity,return_sum,profit,margin,sales_margin
0,Венчик для миксеров KitchenAid,5K5A2WW_DM,5K5A2WW_DM,шт,https://api.moysklad.ru/api/remap/1.2/entity/p...,e430a07d-8ada-11ef-0a80-1006000ec7b5,190.0,4.331248e+05,8.229370e+07,3.605359e+07,8.0,4589599.0,43244115.0,1.254908,0.556523
1,Лопатка с гибким силиконовым ребром для миксер...,KFE5T_DM,KFE5T_DM,шт,https://api.moysklad.ru/api/remap/1.2/entity/p...,b948b7f9-8ada-11ef-0a80-1569000f85d4,75.0,2.150623e+05,1.612968e+07,1.235127e+07,3.0,1574205.0,2698301.0,0.227567,0.185381
2,Машинка для стрижки волос Braun HC5350,Б0062960,742413,шт,https://api.moysklad.ru/api/remap/1.2/entity/p...,a9689274-1f3d-11ef-0a80-054d001b2474,203.0,2.817120e+06,5.718753e+08,4.840354e+08,0.0,0.0,87839959.0,0.181474,0.153600
3,Набор электрических зубных щеток Oral-B Vitali...,80368953,80368953,шт,https://api.moysklad.ru/api/remap/1.2/entity/p...,caec0785-baca-11ee-0a80-00a0000a83bf,2.0,1.893253e+06,3.786505e+06,7.286000e+06,0.0,0.0,-3499495.0,-0.480304,-0.924202
4,Насадка для стрижки бороды Braun XT20 Black дл...,7500435205979,742542,шт,https://api.moysklad.ru/api/remap/1.2/entity/p...,a97b2dab-1f3d-11ef-0a80-054d001b248c,1.0,1.131729e+06,1.131729e+06,6.352000e+05,0.0,0.0,496529.0,0.781689,0.438735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Эпилятор Braun Silk-epil 5 SE5-011 тип 5392,7500435225052,742625,шт,https://api.moysklad.ru/api/remap/1.2/entity/p...,a9c7e2ea-1f3d-11ef-0a80-054d001b24ec,259.0,3.819924e+06,9.893604e+08,6.934771e+08,1.0,4678702.0,293865599.0,0.425389,0.298437
86,Эпилятор Braun Silk-epil 5 SE5-050 FLMG тип 5392,7500435225038,742704,шт,https://api.moysklad.ru/api/remap/1.2/entity/p...,a9efa4ad-1f3d-11ef-0a80-054d001b251c,316.0,4.402637e+06,1.391233e+09,8.917406e+08,1.0,5497888.0,497233739.0,0.559632,0.358823
87,Эпилятор Braun Silk-epil 7 SE7-000 FLMG тип 5392,7500435225151,742623,шт,https://api.moysklad.ru/api/remap/1.2/entity/p...,a9aaf7ae-1f3d-11ef-0a80-054d001b24c8,507.0,4.636852e+06,2.350884e+09,1.738331e+09,4.0,23105264.0,603084636.0,0.349676,0.259082
88,Эпилятор Braun Silk-epil 9 SES9-011 3D тип 538...,7500435225328,742702,шт,https://api.moysklad.ru/api/remap/1.2/entity/p...,aa0372a2-1f3d-11ef-0a80-054d001b2534,424.0,6.421582e+06,2.722751e+09,2.288610e+09,2.0,17730224.0,427075554.0,0.187483,0.157883
